In [1]:
import numpy as np, pandas as pd
import gc
import pickle

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score

pd.set_option('max_columns', None)

In [2]:
spring_2019 = pd.read_excel('2019-spring-2019-05-21.xlsx')
summer_2019 = pd.read_excel('2019-summer-2019-07-08.xlsx')
#summer_2018 = pd.read_excel('2018-summer-2018-09-27.xlsx')
#spring_2018 = pd.read_excel('2018-spring-2018-05-20.xlsx')

## main functions

In [83]:
start_elo = {"LCK" : 1330, 
             "LEC": 1320, 
             "LPL" : 1330,
             "CBLoL" : 1100,
             "LMS" : 1310,
             "LCS" : 1310,
             }

In [84]:
def inc_dec_diff(diff):
    if diff < 24:
        return 16, 16
    elif diff < 49:
        return 15, 17
    elif diff < 74:
        return 14, 18
    elif diff < 99:
        return 13, 19
    elif diff < 124:
        return 12, 20
    elif diff < 149:
        return 11, 21
    elif diff < 174:
        return 10, 22
    elif diff < 199:
        return 9, 23
    elif diff < 224:
        return 8, 24
    elif diff < 249:
        return 7, 25
    elif diff < 274:
        return 6, 26
    elif diff < 299:
        return 5, 27
    else: 
        return 4, 28

In [85]:
info_columns = ['team', 'rival']

def get_features_by_pairs(df, pairs):
    print('startred shape', df.shape)
    df = df.loc[df.url.notnull()]
    df = df.applymap(lambda x: np.nan if x == ' ' else x)
    df.sort_values(['date', 'gameid', 'playerid'], inplace=True)
    print('shape after cleaning', df.shape)
    print('number of games', len(np.unique(df['date'].values)))
    
    rivals = get_rivals(df)
    df = df.merge(rivals, on=['date', 'gameid', 'team'])
    pairwise_winrates = calc_pairwise_winrate(df, pairs)
    
    teams = np.unique(match_schedule)
    mean_features_by_teams = calc_mean_features_by_teams(df, teams)
    pairwise_mean_features = calc_pairwise_mean_features(df, pairs)
    #elo = calc_elo_rating(df)
    
    full_df = pd.DataFrame()
    for ind, (team_1, team_2) in enumerate(pairs):
        tmp_1 = mean_features_by_teams[mean_features_by_teams['team'] == team_1]
        tmp_2 = mean_features_by_teams[mean_features_by_teams['team'] == team_2].rename(columns={'team': 'rival'})
        pair_tmp = pairwise_winrates.merge(tmp_1, on=['team'])
        pair_tmp = pair_tmp.merge(tmp_2, on=['rival'])
        #pair_tmp = pair_tmp.merge(elo[['team', 'elo']], on=['team'])
        #pair_tmp = pair_tmp.merge(elo[['team', 'elo']].rename(columns={'team': 'rival'}), on=['rival'])
        
        full_df = full_df.append(pair_tmp)
        
    full_df = full_df.merge(pairwise_mean_features, on=['team', 'rival'])
    
    return full_df, mean_features_by_teams

In [86]:
def get_rivals(matches_data):
    tmp = matches_data.groupby(['date', 'gameid', 'team']).agg({'result': 'count'}).reset_index()
    
    rivals = []
    for i in range(len(tmp)):
        if i % 2 == 0:
            match_rival = tmp.at[i+1, 'team']
        else:
            match_rival = tmp.at[i-1, 'team']
        rivals.append(match_rival)
    
    tmp['rival'] = rivals
    return tmp[['date', 'gameid', 'team', 'rival']]

In [87]:
def calc_pairwise_winrate(matches_data, pairs):
    pairwise_winrates = []
    matches_results = matches_data.loc[matches_data['position'] == 'Team', ['gameid', 'team', 'rival', 'result']]
    
    for team_1, team_2 in pairs:
        tmp = matches_results[(matches_results['team'] == team_1) & (matches_results['rival'] == team_2)]

        if len(tmp) == 0:
            winrate = np.nan
        else:
            winrate = tmp['result'].mean()
            
        pairwise_winrates.append({
            'team': team_1,
            'rival': team_2,
            'pairwise_winrate': winrate
        })
    
    pairwise_winrates = pd.DataFrame(pairwise_winrates)
    return pairwise_winrates[['team', 'rival', 'pairwise_winrate']]

In [88]:
def calc_mean_features_by_teams(matches_data, teams):
    mean_features = []
    names_of_features = ['kpm', 'okpm', 'teamtowerkills', 'opptowerkills',
                         'dmgtochampsperminute', 'wpm', 'wcpm', 'cspm',
                         'minionkills', 'monsterkillsownjungle', 'monsterkillsenemyjungle',
                         'result',]
    teams_data = matches_data[matches_data['position'] == 'Team']
    
    for team in teams:
        tmp = teams_data[teams_data['team'] == team]
        
        tmp_agg = {}
        if len(tmp) == 0:
            for key in names_of_features:
                tmp_agg[key] = None
        else:
            for key in names_of_features:
                tmp_agg[key] = np.nanmean(tmp[key])

        tmp_agg['team'] = team
        mean_features.append(tmp_agg)
            
    mean_features = pd.DataFrame(mean_features)
    mean_features.rename(columns={'result': 'winrate'}, inplace=True)
    return mean_features

In [89]:
def calc_pairwise_mean_features(matches_data, pairs):
    mean_features = []
    names_of_features = ['dmgtochampsperminute', 'wpm', 'wcpm',
                         'earnedgpm', 'minionkills', 'monsterkills',
                         'monsterkillsownjungle', 'monsterkillsenemyjungle',]
    players_data = matches_data[matches_data['position'] != 'Team']
    unique_teams = np.unique(pairs)
    unique_lanes = np.unique(players_data.position.values)

    for team in unique_teams:
        tmp = players_data[players_data['team'] == team]

        tmp_agg = {}
        if len(tmp) == 0:
            for lane in unique_lanes:
                for key in names_of_features:
                    tmp_agg[key+'_'+lane.lower()] = None
        else:
            for lane in unique_lanes:
                tmp_by_lane = tmp[tmp['position'] == lane]
                for key in names_of_features:
                    tmp_agg[key+'_'+lane.lower()] = np.nanmean(tmp_by_lane[key])

        tmp_agg['team'] = team
        mean_features.append(tmp_agg)

    mean_features = pd.DataFrame(mean_features)
    
    features = list(set(mean_features.columns) - set(['team']))

    pairwise_statistics = []
    for team_1, team_2 in pairs:
        team_1_data = mean_features[mean_features['team'] == team_1]
        team_2_data = mean_features[mean_features['team'] == team_2]

        statistics_by_features = {}
        for f in features:
            statistics_by_features[f+'_pairwise'] = team_1_data[f].values[0] / (team_2_data[f].values[0] + 1)

        statistics_by_features['team'] = team_1
        statistics_by_features['rival'] = team_2
        pairwise_statistics.append(statistics_by_features)
    
    pairwise_statistics = pd.DataFrame(pairwise_statistics)

    return pairwise_statistics

In [90]:
def calc_elo_rating(df):
    elo = df[["team", "league"]].drop_duplicates()
    elo.drop_duplicates("team", inplace=True)
    elo["elo"] = 0
    elo["elo"] = elo["league"].map(start_elo)
    
    for team in elo.loc[elo.league == "MSI", "team"]:
        elo_in_league = elo.loc[(elo.league != "MSI")][elo.team == team]
    if elo_in_league.empty:
        elo.loc[(elo.league == "MSI") & (elo.team == team), "elo"] = 1100
    else:
        elo.loc[(elo.league == "MSI") & (elo.team == team), "elo"] = elo_in_league.elo.values
    elo.fillna(1000, inplace=True)
    df = df.drop_duplicates("date")
    df = df.sort_values(['date', 'gameid'])
    
    #df["percent with Elo"] = 0
    df["team_elo"] = 0
    df["rival_elo"] = 0
    for _, game in df.iterrows():
        diff = np.abs(elo.loc[elo.team == game.team, "elo"].values[0] 
                      - elo.loc[elo.team == game.rival, "elo"].values[0]) 
        inc,  dec = inc_dec_diff(diff)
        elo.loc[elo.team == game.team, "elo"] += inc
        elo.loc[elo.team == game.rival, "elo"] -= dec
        if game.result == 1:
            elo.loc[elo.team == game.team, "elo"] += inc
            elo.loc[elo.team == game.rival, "elo"] -= dec
        else:
            elo.loc[elo.team == game.team, "elo"] -= dec
            elo.loc[elo.team == game.rival, "elo"] += inc
        df.loc[(df.gameid == game.gameid) & (df.team == game.team), "team_elo"] = elo.loc[elo.team == game.team, "elo"].values[0]
        df.loc[(df.gameid == game.gameid) & (df.team == game.team), "rival_elo"] = elo.loc[elo.team == game.rival, "elo"].values[0]
    #df.loc[(df.gameid == game.gameid) & (df.team == game.team), "percent with Elo"] = 1 / (10 ** (-diff/400) + 1)
    #return df[["team", "rival", "percent with Elo", "date"]]
    #return df[["team", "rival", "team_elo", "rival_elo", "date"]]
    return elo

In [91]:
def swap_pairs(pairs):
    all_pairs = []
    for pair in pairs:
        all_pairs.append(pair)
        all_pairs.append([pair[1], pair[0]])

    return all_pairs

## get features

In [10]:
summer_lcs = summer_2019[summer_2019['league'] == 'LCS']
summer_lec = summer_2019[summer_2019['league'] == 'LEC']
df_for_predictions = summer_lcs.append(summer_lec)

match_schedule = [['Cloud9', 'G2 Esports'],
                  ['Team Liquid', 'Fnatic'],
                  ['Team SoloMid', 'Origen'],
                  ['Cloud9', 'Fnatic']]
match_schedule = swap_pairs(match_schedule)

features_df, mean_features_by_teams = get_features_by_pairs(df_for_predictions, match_schedule)
X_test = features_df.drop(info_columns, axis=1)

startred shape (840, 98)
shape after cleaning (840, 98)
number of games 70


In [11]:
features_df.head()

,team,rival,pairwise_winrate,cspm_x,dmgtochampsperminute_x,kpm_x,minionkills_x,monsterkillsenemyjungle_x,monsterkillsownjungle_x,okpm_x,opptowerkills_x,winrate_x,teamtowerkills_x,wcpm_x,wpm_x,cspm_y,dmgtochampsperminute_y,kpm_y,minionkills_y,monsterkillsenemyjungle_y,monsterkillsownjungle_y,okpm_y,opptowerkills_y,winrate_y,teamtowerkills_y,wcpm_y,wpm_y,dmgtochampsperminute_adc_pairwise,dmgtochampsperminute_jungle_pairwise,dmgtochampsperminute_middle_pairwise,dmgtochampsperminute_support_pairwise,dmgtochampsperminute_top_pairwise,earnedgpm_adc_pairwise,earnedgpm_jungle_pairwise,earnedgpm_middle_pairwise,earnedgpm_support_pairwise,earnedgpm_top_pairwise,minionkills_adc_pairwise,minionkills_jungle_pairwise,minionkills_middle_pairwise,minionkills_support_pairwise,minionkills_top_pairwise,monsterkills_adc_pairwise,monsterkills_jungle_pairwise,monsterkills_middle_pairwise,monsterkills_support_pairwise,monsterkills_top_pairwise,monsterkillsenemyjungle_adc_pairwise,monsterkillsenemyjungle_jungle_pairwise,monsterkillsenemyjungle_middle_pairwise,monsterkillsenemyjungle_support_pairwise,monsterkillsenemyjungle_top_pairwise,monsterkillsownjungle_adc_pairwise,monsterkillsownjungle_jungle_pairwise,monsterkillsownjungle_middle_pairwise,monsterkillsownjungle_support_pairwise,monsterkillsownjungle_top_pairwise,wcpm_adc_pairwise,wcpm_jungle_pairwise,wcpm_middle_pairwise,wcpm_support_pairwise,wcpm_top_pairwise,wpm_adc_pairwise,wpm_jungle_pairwise,wpm_middle_pairwise,wpm_support_pairwise,wpm_top_pairwise
0,Cloud9,G2 Esports,NaN,32.460888,1838.301321,0.413854,929.125000,18.125000,129.500000,0.310185,5.125000,0.625000,7.25,1.600743,3.555841,31.958857,2347.086158,0.722060,688.333333,19.333333,106.000000,0.468011,3.166667,0.833333,8.000000,1.348418,3.344558,0.751057,0.889661,0.880307,0.463981,0.841694,0.745804,0.921883,0.894199,0.902624,1.053602,1.100141,1.386792,1.257336,4.080275,1.459924,0.942708,1.286179,0.854651,0.0,1.335616,0.505814,0.739130,0.988636,0.0,2.375000,1.141304,1.296380,0.757732,0.0,0.995192,0.310592,0.318543,0.196526,0.242539,0.166362,0.330896,0.377063,0.325159,0.639212,0.363880
1,G2 Esports,Cloud9,NaN,31.958857,2347.086158,0.722060,688.333333,19.333333,106.000000,0.468011,3.166667,0.833333,8.00,1.348418,3.344558,32.460888,1838.301321,0.413854,929.125000,18.125000,129.500000,0.310185,5.125000,0.625000,7.250000,1.600743,3.555841,1.326507,1.116950,1.131416,2.134240,1.183246,1.332203,1.075090,1.110628,1.092016,0.942377,0.901660,0.675497,0.788741,0.228508,0.679114,0.973545,0.765577,1.058065,0.0,0.647343,1.333333,1.105263,0.576577,0.0,0.000000,0.743363,0.753022,1.144654,0.0,0.796537,0.215640,0.322276,0.148590,0.206509,0.100076,0.279213,0.558430,0.266460,0.486055,0.276061
2,Team Liquid,Fnatic,NaN,32.927799,1941.117256,0.426567,867.750000,27.875000,113.000000,0.249122,4.000000,0.750000,8.25,1.519891,3.108810,33.234535,1852.508784,0.565768,845.666667,27.333333,110.666667,0.311135,2.666667,1.000000,9.000000,1.413805,3.376790,1.459402,0.757538,0.862496,0.611803,1.333752,0.938817,1.038704,0.781752,0.859224,0.925745,0.882733,1.881944,1.028545,1.352848,1.091111,1.136943,1.125532,0.573529,0.0,0.500000,1.783784,0.739286,0.570652,0.0,0.664286,0.725806,1.193262,0.697674,0.0,0.366279,0.259817,0.331456,0.137706,0.313585,0.117595,0.289475,0.300663,0.256008,0.596432,0.329211
3,Fnatic,Team Liquid,NaN,33.234535,1852.508784,0.565768,845.666667,27.333333,110.666667,0.311135,2.666667,1.000000,9.00,1.413805,3.376790,32.927799,1941.117256,0.426567,867.750000,27.875000,113.000000,0.249122,4.000000,0.750000,8.250000,1.519891,3.108810,0.682352,1.309778,1.154786,1.615513,0.746446,1.058787,0.953790,1.271169,1.147379,1.072397,1.125076,0.487455,0.965210,0.691695,0.908722,0.818428,0.874296,1.619586,0.0,1.707317,0.430556,1.108225,1.240310,0.0,0.991453,1.183673,0.816446,1.281250,0.0,1.701149,0.296091,0.229476,0.156438,0.222646,0.160385,0.346254,0.472451,0.370574,0.499090,0.343995
4,Team SoloMid,Origen,NaN,31.772590,1868.995746,0.359123,931.750000,13.250000,128.125000,0.2

In [12]:
X_test.head()

,pairwise_winrate,cspm_x,dmgtochampsperminute_x,kpm_x,minionkills_x,monsterkillsenemyjungle_x,monsterkillsownjungle_x,okpm_x,opptowerkills_x,winrate_x,teamtowerkills_x,wcpm_x,wpm_x,cspm_y,dmgtochampsperminute_y,kpm_y,minionkills_y,monsterkillsenemyjungle_y,monsterkillsownjungle_y,okpm_y,opptowerkills_y,winrate_y,teamtowerkills_y,wcpm_y,wpm_y,dmgtochampsperminute_adc_pairwise,dmgtochampsperminute_jungle_pairwise,dmgtochampsperminute_middle_pairwise,dmgtochampsperminute_support_pairwise,dmgtochampsperminute_top_pairwise,earnedgpm_adc_pairwise,earnedgpm_jungle_pairwise,earnedgpm_middle_pairwise,earnedgpm_support_pairwise,earnedgpm_top_pairwise,minionkills_adc_pairwise,minionkills_jungle_pairwise,minionkills_middle_pairwise,minionkills_support_pairwise,minionkills_top_pairwise,monsterkills_adc_pairwise,monsterkills_jungle_pairwise,monsterkills_middle_pairwise,monsterkills_support_pairwise,monsterkills_top_pairwise,monsterkillsenemyjungle_adc_pairwise,monsterkillsenemyjungle_jungle_pairwise,monsterkillsenemyjungle_middle_pairwise,monsterkillsenemyjungle_support_pairwise,monsterkillsenemyjungle_top_pairwise,monsterkillsownjungle_adc_pairwise,monsterkillsownjungle_jungle_pairwise,monsterkillsownjungle_middle_pairwise,monsterkillsownjungle_support_pairwise,monsterkillsownjungle_top_pairwise,wcpm_adc_pairwise,wcpm_jungle_pairwise,wcpm_middle_pairwise,wcpm_support_pairwise,wcpm_top_pairwise,wpm_adc_pairwise,wpm_jungle_pairwise,wpm_middle_pairwise,wpm_support_pairwise,wpm_top_pairwise
0,NaN,32.460888,1838.301321,0.413854,929.125000,18.125000,129.500000,0.310185,5.125000,0.625000,7.25,1.600743,3.555841,31.958857,2347.086158,0.722060,688.333333,19.333333,106.000000,0.468011,3.166667,0.833333,8.000000,1.348418,3.344558,0.751057,0.889661,0.880307,0.463981,0.841694,0.745804,0.921883,0.894199,0.902624,1.053602,1.100141,1.386792,1.257336,4.080275,1.459924,0.942708,1.286179,0.854651,0.0,1.335616,0.505814,0.739130,0.988636,0.0,2.375000,1.141304,1.296380,0.757732,0.0,0.995192,0.310592,0.318543,0.196526,0.242539,0.166362,0.330896,0.377063,0.325159,0.639212,0.363880
1,NaN,31.958857,2347.086158,0.722060,688.333333,19.333333,106.000000,0.468011,3.166667,0.833333,8.00,1.348418,3.344558,32.460888,1838.301321,0.413854,929.125000,18.125000,129.500000,0.310185,5.125000,0.625000,7.250000,1.600743,3.555841,1.326507,1.116950,1.131416,2.134240,1.183246,1.332203,1.075090,1.110628,1.092016,0.942377,0.901660,0.675497,0.788741,0.228508,0.679114,0.973545,0.765577,1.058065,0.0,0.647343,1.333333,1.105263,0.576577,0.0,0.000000,0.743363,0.753022,1.144654,0.0,0.796537,0.215640,0.322276,0.148590,0.206509,0.100076,0.279213,0.558430,0.266460,0.486055,0.276061
2,NaN,32.927799,1941.117256,0.426567,867.750000,27.875000,113.000000,0.249122,4.000000,0.750000,8.25,1.519891,3.108810,33.234535,1852.508784,0.565768,845.666667,27.333333,110.666667,0.311135,2.666667,1.000000,9.000000,1.413805,3.376790,1.459402,0.757538,0.862496,0.611803,1.333752,0.938817,1.038704,0.781752,0.859224,0.925745,0.882733,1.881944,1.028545,1.352848,1.091111,1.136943,1.125532,0.573529,0.0,0.500000,1.783784,0.739286,0.570652,0.0,0.664286,0.725806,1.193262,0.697674,0.0,0.366279,0.259817,0.331456,0.137706,0.313585,0.117595,0.289475,0.300663,0.256008,0.596432,0.329211
3,NaN,33.234535,1852.508784,0.565768,845.666667,27.333333,110.666667,0.311135,2.666667,1.000000,9.00,1.413805,3.376790,32.927799,1941.117256,0.426567,867.750000,27.875000,113.000000,0.249122,4.000000,0.750000,8.250000,1.519891,3.108810,0.682352,1.309778,1.154786,1.615513,0.746446,1.058787,0.953790,1.271169,1.147379,1.072397,1.125076,0.487455,0.965210,0.691695,0.908722,0.818428,0.874296,1.619586,0.0,1.707317,0.430556,1.108225,1.240310,0.0,0.991453,1.183673,0.816446,1.281250,0.0,1.701149,0.296091,0.229476,0.156438,0.222646,0.160385,0.346254,0.472451,0.370574,0.499090,0.343995
4,NaN,31.772590,1868.995746,0.359123,931.750000,13.250000,128.125000,0.296613,5.500000,0.625000,7.00,1.219875,3.528686,32.629650,1797.161845,0.376346,938.000000,13.833333,120.50

In [13]:
mean_features_by_teams.head()

,cspm,dmgtochampsperminute,kpm,minionkills,monsterkillsenemyjungle,monsterkillsownjungle,okpm,opptowerkills,winrate,team,teamtowerkills,wcpm,wpm
0,32.460888,1838.301321,0.413854,929.125000,18.125000,129.500000,0.310185,5.125000,0.625000,Cloud9,7.250000,1.600743,3.555841
1,33.234535,1852.508784,0.565768,845.666667,27.333333,110.666667,0.311135,2.666667,1.000000,Fnatic,9.000000,1.413805,3.376790
2,31.958857,2347.086158,0.722060,688.333333,19.333333,106.000000,0.468011,3.166667,0.833333,G2 Esports,8.000000,1.348418,3.344558
3,32.629650,1797.161845,0.376346,938.000000,13.833333,120.500000,0.334267,6.000000,0.500000,Origen,6.166667,1.553182,3.697475
4,32.927799,1941.117256,0.426567,867.750000,27.875000,113.000000,0.249122,4.000000,0.750000,Team Liquid,8.250000,1.519891,3.108810


## get predictions

In [72]:
df_for_predictions = spring_2019.append(summer_2019)

match_schedule = [['Cloud9', 'G2 Esports'],
                  ['Team Liquid', 'Fnatic'],
                  ['Team SoloMid', 'Origen'],
                  ['Cloud9', 'Fnatic']]
match_schedule = swap_pairs(match_schedule)

In [73]:
features_df, mean_features_by_teams = get_features_by_pairs(df_for_predictions, match_schedule)
X_test = features_df.drop(info_columns, axis=1)

startred shape (16488, 98)
shape after cleaning (11676, 98)
number of games 973


In [75]:
with open('model_0407_01.pkl', 'rb') as model_pkl:
    lgb_clf = pickle.load(model_pkl)  # init model
preds = lgb_clf.predict(X_test)

print('DATA: {}'.format('summer lcs/lec'))
print('shape: {}'.format(df_for_predictions.shape))
for ind, _ in enumerate(preds):
    if ind % 2 == 0:
        team_1, team_2 = match_schedule[ind]
        argmax_prediction = np.argmax(preds[ind:ind+2])
        if argmax_prediction == 0:
            print('{0} VS {1} prediction {2:.3}'.format(team_1, team_2, preds[ind]))
        elif argmax_prediction == 1:
            print('{0} VS {1} prediction {2:.3}'.format(team_2, team_1, preds[ind+1]))
            
print()
print('ALL PREDICTIONS')
for (team_1, team_2), prediction in zip(match_schedule, preds):
    print('{0} VS {1} prediction {2:.3}'.format(team_1, team_2, prediction))

DATA: summer lcs/lec
shape: (16488, 98)
G2 Esports VS Cloud9 prediction 0.582
Fnatic VS Team Liquid prediction 0.56
Origen VS Team SoloMid prediction 0.553
Fnatic VS Cloud9 prediction 0.637

ALL PREDICTIONS
Cloud9 VS G2 Esports prediction 0.458
G2 Esports VS Cloud9 prediction 0.582
Team Liquid VS Fnatic prediction 0.502
Fnatic VS Team Liquid prediction 0.56
Team SoloMid VS Origen prediction 0.541
Origen VS Team SoloMid prediction 0.553
Cloud9 VS Fnatic prediction 0.445
Fnatic VS Cloud9 prediction 0.637
